# Playground for ASFPy module development

In [1]:
import os

from asfpy import asfpy

In [2]:
categories = [
    "Clinical",
    "Developmental",
    "Affective",
    "Social",
    "Cognitive"
]

In [40]:
applicants = [
    {
        "id": "APP001",
        "urm": False,
        "lim": False,
        "du": True,
        "is_flexible": True,
        "categories": {
            "Clinical"
        },
    },
    {
        "id": "APP002",
        "urm": True,
        "lim": False,
        "du": True,
        "is_flexible": True,
        "categories": {
            "Cognitive"
        },
    },
    {
        "id": "APP003",
        "urm": True,
        "lim": False,
        "du": True,
        "is_flexible": True,
        "categories": {
            "Social",
            "Cognitive"
        }
    },
    {
        "id": "APP004",
        "urm": False,
        "lim": False,
        "du": True,
        "is_flexible": True,
        "categories": {
            "Clinical"
        },
    },
    {
        "id": "APP005",
        "urm": True,
        "lim": True,
        "du": True,
        "is_flexible": True,
        "categories": {
            "Cognitive",
            "Affective"
        },
    },
    {
        "id": "APP006",
        "urm": True,
        "lim": False,
        "du": True,
        "is_flexible": True,
        "categories": {
            "Affective",
            "Cognitive"
        }
    },
    {
        "id": "APP007",
        "urm": True,
        "lim": True,
        "du": False,
        "is_flexible": False,        
        "categories": {
            "Clinical"
        }
    },
    {
        "id": "APP008",
        "urm": False,
        "lim": True,
        "du": False,
        "is_flexible": True,        
        "categories": {
            "Developmental"
        }
    },
    {
        "id": "APP009",
        "urm": True,
        "lim": True,
        "du": False,
        "is_flexible": False,        
        "categories": {
            "Clinical"
        }
    },
    {
        "id": "APP010",
        "urm": False,
        "lim": True,
        "du": False,
        "is_flexible": True,        
        "categories": {
            "Affective"
        }
    },
]

In [74]:
editors = [
    {
        "id": "EDI001",
        "role": "faculty",
        "categories": {
            "Clinical",
            "Developmental"
        },
        "capacity": 2
    },
    {
        "id": "EDI002",
        "role": "faculty",
        "categories": {
            "Affective"
        },
        "capacity": 2
    },
    {
        "id": "EDI003",
        "role": "faculty",
        "categories": {
            "Cognitive",
            "Affective"
        },
        "capacity": 5
    },
    {
        "id": "EDI004",
        "role": "student",
        "categories": {
            "Clinical",
            "Developmental"
        },
        "capacity": 2
    },
    {
        "id": "EDI005",
        "role": "student",
        "categories": {
            "Cognitive",
            "Social"
        },
        "capacity": 8
    },
    {
        "id": "EDI006",
        "role": "student",
        "categories": {
            "Clinical",
            "Developmental"
        },
        "capacity": 5
    }
]

In [91]:
import operator

def find_match(applicant, editors):
    """
    Match an applicant to editors, if possible.
    """
    if asfpy.capacity(editors) > 0:
    # If at least one editor in a list is available for an applicant,
    # find the best possible match and assign.

        highest_capacity_category = asfpy.find_highest_capacity_category(applicant, editors)

        highest_capacity_editors = sorted(
            asfpy.editors_by_categories(editors, {highest_capacity_category}), 
            key = operator.itemgetter("capacity"),
            reverse = True
        )

        editor_id = highest_capacity_editors[0]["id"]

        return highest_capacity_editors[0]["id"]

    else:
    # If no editors have capacity within the group, return None    
        return None

def update_capacity(editor_id, editors):
    """
    Update capacity of editor within a list by id.
    """
    for editor in editors:
        if editor["id"] == editor_id:
            editor["capacity"] -= 1
    
def allocate(applicants, editors):
    """
    Allocate applicants to editors.
    """
    unmatched = [applicant["id"] for applicant in applicants]
    matchings = []
    
    for applicant in applicants:
        
        potential_editors = asfpy.editors_by_categories(editors, applicant["categories"])
        
        if asfpy.capacity(potential_editors) < 2:
        # If the editing capacity for an applicant is less than 2, continue to next applicant    
            continue
        else:
            
            _match = {
                "applicant": applicant["id"],
                "editors": []
            }
            
            faculty_editors = asfpy.editors_by_role(potential_editors, "faculty")
            student_editors = asfpy.editors_by_role(potential_editors, "student")
            
            faculty_editor_match = find_match(applicant, faculty_editors)
            
            if (faculty_editor_match is not None) and (asfpy.capacity(student_editors) > 0):
            # If a faculty editor match is possible and at least one student editor
            # match is possible, then add a faculty editor match and update capacity.
                _match["editors"].append(faculty_editor_match)
                update_capacity(faculty_editor_match, faculty_editors)
            else:
                if asfpy.capacity(student_editors) < 2:
                # If fewer than 2 student editors are available, skip to next
                # applicant.
                    continue
                elif applicant["is_flexible"]:
                # If an applicant is flexible and prefers to be matched with two
                # student editors, find first match here.
                    student_editor_match = find_match(applicant, student_editors)
                    if student_editor_match is not None:
                        _match["editors"].append(student_editor_match)
                        update_capacity(student_editor_match, student_editors)
                else:
                # If applicant prefers not to have a match if at least one faculty
                # editor is not available, then continue to next applicant.
                    continue
            
            # Add a second editor: A student editor match. Then update
            # capacity of that editor.
            student_editor_match = find_match(applicant, student_editors)
            _match["editors"].append(student_editor_match)
            update_capacity(student_editor_match, student_editors)
            
            # Append the match to matchings and remove this applicant
            # from the list of unmatched applicants.
            matchings.append(_match)
            unmatched.remove(applicant["id"])
            
    return {
        "matchings": matchings,
        "unmatched": unmatched
    }

In [90]:
from copy import deepcopy
_editors = deepcopy(editors)

p_applicants = asfpy.prioritize(applicants, rank_method = asfpy.asfp_rank)

allocate(p_applicants, _editors)

{'matchings': [{'applicant': 'APP005', 'editors': ['EDI003', 'EDI005']},
  {'applicant': 'APP007', 'editors': ['EDI001', 'EDI006']},
  {'applicant': 'APP009', 'editors': ['EDI001', 'EDI006']},
  {'applicant': 'APP002', 'editors': ['EDI003', 'EDI005']},
  {'applicant': 'APP003', 'editors': ['EDI003', 'EDI005']},
  {'applicant': 'APP006', 'editors': ['EDI002', 'EDI005']},
  {'applicant': 'APP008', 'editors': ['EDI006', 'EDI004']},
  {'applicant': 'APP001', 'editors': ['EDI006', 'EDI004']}],
 'unmatched': ['APP010', 'APP004']}